In [ ]:
# import the needed Python modules

import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from numpy import argmax

In [ ]:
# load the MNIST dataset for DL tranining
# . X is the input; y is the expected result
# . two sets -- one for traning; one for validation

(X_train, y_train), (X_valid, y_valid) = mnist.load_data()
print("the shapes of the datasets")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_valid:", X_valid.shape)
print("y_valid:", y_valid.shape)

In [ ]:
# show the first 12 training digits

plt.figure(figsize=(5,5))
for k in range(12):
    plt.subplot(3, 4, k+1)
    plt.imshow(X_train[k], cmap='Greys')
    plt.axis('off')
plt.tight_layout()
plt.show()
print("the first 12 traninig digits:", y_train[0:12])

In [ ]:
# shoe the first one for validation
plt.imshow(X_valid[0], cmap='Greys')

In [ ]:
# save the original valiation dataset for later use
ORI_X_valid = X_valid
ORI_y_valid = y_valid

In [ ]:
# adjust the traning and validation input

X_train = X_train.reshape(60000, 784).astype('float32') / 255
X_valid = X_valid.reshape(10000, 784).astype('float32') / 255
print("X_train:", X_train)
print("X_valid:", X_valid)

In [ ]:
# adjust the traning and validation output

n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_valid = keras.utils.to_categorical(y_valid, n_classes)
print(ORI_y_valid[0], "==>", y_valid[0])  # show the "categorization" for first one

In [ ]:
# build the DL traning model

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
# compile the DL traning model
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.1), metrics=['accuracy'])

In [ ]:
# training (fitting) the model
history = model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(X_valid, y_valid))

In [ ]:
# show the history of the traning (fitting)

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'r.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# try out one with the trained model

idx = 6
plt.imshow(ORI_X_valid[idx], cmap='Greys')
prediction = model.predict(X_valid[idx : (idx + 1)])[0]
ans = argmax(prediction)
print("ANS:", ans)

In [ ]:
# convert and save the model in TFLite format

!mkdir output
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("output/mnist_model.tflite", "wb").write(tflite_model)

In [ ]:
# save the TFLite file as a C source file -- # save the TFLite file as a C source file

!apt-get -qq install xxd
!xxd -i output/mnist_model.tflite > output/mnist_model.h